In [1]:
import time
import numpy as np
import theano
import lasagne
import theano.tensor as T
from helpers.mnist_dataset import load_mnist_rot, test_mnist_rot
from helpers.Models import build_st_network
from helpers.utils import iterate_minibatches

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/ilker/anaconda2/envs/theano/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:601: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Using cuda_convnet (faster)


In [2]:
# Parameters
BATCH_SIZE = 100
EPOCH = 100
BINS = np.linspace(-3, 3, 100)
DISCRETE = True

In [6]:
# Load Dataset
X_train, y_train, X_val, y_val = load_mnist_rot(path='mnist_rotation_new.zip', which='train')
Xtst, Ytst = load_mnist_rot(path='mnist_rotation_new.zip', which='test')

['mnist_all_rotation_normalized_float_train_valid.amat', 'mnist_all_rotation_normalized_float_test.amat']
['mnist_all_rotation_normalized_float_train_valid.amat', 'mnist_all_rotation_normalized_float_test.amat']


In [17]:
# Build the model, and place holders
network = build_st_network(b_size=BATCH_SIZE,
                           bins_choose=BINS,
                           withdiscrete=DISCRETE,
                           input_shape=X_train.shape)

X = T.tensor4('inputs', dtype='float32')
Y = T.ivector('targets')

### Different Cost and outputs
* The cost in update function and train function are using the cont. weights
* The test and eval. functions are using the disc. weights

In [18]:
# Loss is continious
prediction_cont = lasagne.layers.get_output(network, X, deterministic=False, withdiscrete=False)
loss_cont = T.mean(lasagne.objectives.categorical_crossentropy(prediction_cont, Y), dtype='float32')

prediction_disc = lasagne.layers.get_output(network, X, deterministic=False, withdiscrete=True)
loss_disc = T.mean(lasagne.objectives.categorical_crossentropy(prediction_disc, Y), dtype='float32')

In [19]:
# Get parameters, set update rule and create functions
params = lasagne.layers.get_all_params(network, trainable=True)
# We shall use the cont. cost in update function
updates = lasagne.updates.nesterov_momentum(loss_cont, params,
                                            learning_rate=0.01, momentum=0.9)
# Test function
# Test_prediction is on discrete values
test_prediction = lasagne.layers.get_output(network, X, deterministic=True, withdiscrete=True)
test_loss = T.mean(lasagne.objectives.categorical_crossentropy(test_prediction, Y), dtype='float32')
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), Y), dtype='float32')
val_fn = theano.function([X, Y], [test_loss, test_acc], allow_input_downcast=True)

# Train function
train_fn = theano.function([X, Y], loss_disc, updates=updates, allow_input_downcast=True)

ERROR (theano.gof.opt): Optimization failure due to: local_conv2d_gpu_conv
ERROR (theano.gof.opt): node: AbstractConv2d{border_mode='valid', subsample=(1, 1), filter_flip=True, imshp=(None, 32, 12, 12), kshp=(32, 32, 5, 5), filter_dilation=(1, 1)}(Pool{ds=(2, 2), ignore_border=True, st=(2, 2), padding=(0, 0), mode='max'}.0, HostFromGpu.0)
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/ilker/anaconda2/envs/theano/lib/python2.7/site-packages/theano/gof/opt.py", line 1820, in process_node
    replacements = lopt.transform(node)
  File "/home/ilker/anaconda2/envs/theano/lib/python2.7/site-packages/theano/sandbox/cuda/opt.py", line 2745, in local_conv2d_gpu_conv
    inps[0] = as_cuda_ndarray_variable(inps[0])
  File "/home/ilker/anaconda2/envs/theano/lib/python2.7/site-packages/theano/sandbox/cuda/basic_ops.py", line 46, in as_cuda_ndarray_variable
    return gpu_from_host(tensor_x)
  File "/home/ilker/anaconda2/envs/theano/lib/py

In [ ]:
# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(EPOCH):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, BATCH_SIZE, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, BATCH_SIZE, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, EPOCH, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, BATCH_SIZE, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))